### Note: Because i have less compute, i am trying to run this with colab, the model is very big. To make sure i get the results and it do not take days to execute, I have reduced the size of the data. now it takes less than 30 mins to train. The accuracy wont be that good becuase of this reason. if You want better evaluation score. it might not be possible as this course wont be availabe to me after 31st and it is taking more than 3 days to train on the entire dataset.

## Install and import Libraries

In [1]:
pip install transformers datasets peft bitsandbytes torch

In [2]:
!pip install huggingface_hub

In [3]:
from huggingface_hub import login
login()

In [4]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb


## Load Dataset and split (For faster demonstration i am taking very less training data otherwise it was showing 3 days for training)

In [12]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset
dataset = load_dataset("finGPT/fingpt-sentiment-train")
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 76772
    })
})

In [5]:
# Split the dataset into train and eval
train_size = int(0.005 * len(dataset['train']))
eval_size = int((len(dataset['train']) - train_size) *0.01)
train_dataset = dataset['train'].select(range(train_size))
eval_dataset = dataset['train'].select(range(train_size, train_size + eval_size))

# Create a new DatasetDict with train and eval splits
split_dataset = DatasetDict({
    'train': train_dataset,
    'eval': eval_dataset
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


## Load tokenizer and preprocess data

In [6]:
# Load Mistral 7B model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define label mapping
label_map = {
    'mildly negative': 0,
    'mildly positive': 1,
    'moderately negative': 2,
    'moderately positive': 3,
    'negative': 4,
    'neutral': 5,
    'positive': 6,
    'strong negative': 7,
    'strong positive': 8
}

reverse_label_map = {v: k for k, v in label_map.items()}


# Preprocessing function
def preprocess_function(examples):
    # Combine instruction and input for context
    texts = [f"{instr} {inp}" for instr, inp in zip(examples['instruction'], examples['input'])]

    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Convert sentiment labels from output to numerical values
    labels = [label_map[out] for out in examples['output']]

    encodings['labels'] = labels
    return encodings

# Preprocess the datasets
tokenized_dataset = split_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['input', 'output', 'instruction']
)

Map:   0%|          | 0/763 [00:00<?, ? examples/s]

## Load model

In [7]:
# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Define LoRa config

In [8]:
# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


## Fine tune model

In [9]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./mistral-sentiment-lora",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=500,
    logging_steps=100,
    warmup_steps=100,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps",
    eval_steps=500,
    report_to="none"
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    data_collator=data_collator,
)

# Fine-tune the model with LoRA
trainer.train()

# Save the fine-tuned model and LoRA adapters
model.save_pretrained("./mistral-sentiment-lora-finetuned")
tokenizer.save_pretrained("./mistral-sentiment-lora-finetuned")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between 

Step,Training Loss,Validation Loss


('./mistral-sentiment-lora-finetuned/tokenizer_config.json',
 './mistral-sentiment-lora-finetuned/special_tokens_map.json',
 './mistral-sentiment-lora-finetuned/tokenizer.model',
 './mistral-sentiment-lora-finetuned/added_tokens.json',
 './mistral-sentiment-lora-finetuned/tokenizer.json')

## Evaluate Model

In [10]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 1.836306095123291, 'eval_runtime': 492.7857, 'eval_samples_per_second': 1.548, 'eval_steps_per_second': 0.388, 'epoch': 1.0}
